In [ ]:
!pip install "sagemaker>=2.123.0"

In [ ]:
import sagemaker
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from datetime import datetime

# Get default bucket
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlops-workshop'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

# SageMaker Session
sagemaker_session = sagemaker.session.Session()

In [ ]:
%store -r

In [ ]:
print(train_uri)
print(test_uri)
print(val_uri)

In [ ]:
image_uri = sagemaker.image_uris.retrieve(region=region, framework='xgboost', version='latest')

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.experiments import Run

model_path = f"s3://{bucket}/{prefix}/xgb_model"
experiment_name = f'mlops-workshop-experiment'

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=sagemaker_session,
    keep_alive_period_in_seconds=3600
)

with Run(experiment_name = experiment_name, sagemaker_session = sagemaker_session) as run:      
    xgb_train.set_hyperparameters(
        objective="binary:logistic",
        num_round=50,
        max_depth=5,
        eta=0.2,
        gamma=4,
        min_child_weight=6,
        subsample=0.7,
        silent=0
    )

    xgb_train.fit(
        inputs = {
            "train": TrainingInput(
                s3_data=train_uri,
                content_type="text/csv"
            ),
            'validation': TrainingInput(
                s3_data=val_uri,
                content_type="text/csv"
            )
        }
    ) 

In [ ]:
trained_model_uri = xgb_train.model_data

In [ ]:
training_image = xgb_train.image_uri

In [ ]:
%store trained_model_uri
%store training_image